# FARSITE Minimal Demo

**Completely standalone** — no dependencies on parent repo.

**Requirements:**
- `TestFARSITE` executable (in this directory)
- `lcpmake` executable (in this directory)
- `NoBarrier/` shapefile directory (in this directory)
- `landscape.py` and `farsite.py` (in this directory)

**What you'll do:**
1. Create a simple landscape
2. Define an ignition point
3. Run FARSITE
4. Visualize results
5. Explore parameters

## Setup

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import geopandas as gpd

# Import from local scripts
from landscape import create_simple_landscape, verify_lcp_file
from farsite import run_farsite

DEMO_DIR = Path.cwd()
print(f"Demo directory: {DEMO_DIR}")

ModuleNotFoundError: No module named 'osgeo'

## 1. Create Landscape

In [ ]:
lcp_path = DEMO_DIR / 'simple_landscape.lcp'

create_simple_landscape(
    output_path=lcp_path,
    size_km=10,
    resolution_m=30,
    fuel_model=1,           # Short grass
    elevation_m=1000,
    slope_pct=10,
    canopy_cover_pct=0,     # No canopy
    latitude=34.0,
    verbose=True
)

In [ ]:
verify_lcp_file(lcp_path, verbose=True)

## 2. Define Ignition

In [ ]:
# Center of landscape (NAD83 Albers)
ignition_center = Point(505000, 4005000)
ignition_poly = ignition_center.buffer(100)  # 100m radius

print(f"Ignition: 100m circle at (505000, 4005000)")
print(f"Area: {ignition_poly.area / 1e6:.4f} km²")

## 3. Run FARSITE

In [ ]:
result = run_farsite(
    ignition_polygon=ignition_poly,
    lcp_path=str(lcp_path),
    windspeed=15,
    winddirection=270,      # Westerly (blows east)
    duration_hours=6,
    dist_res=30,
    perim_res=60,
    verbose=True
)

## 4. Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot ignition
gpd.GeoDataFrame(geometry=[ignition_poly], crs="EPSG:5070").plot(
    ax=ax, color='red', edgecolor='darkred', linewidth=2, label='Ignition'
)

# Plot result
if result:
    gpd.GeoDataFrame(geometry=[result], crs="EPSG:5070").plot(
        ax=ax, facecolor='orange', edgecolor='black',
        alpha=0.6, linewidth=2, label='Final (6 hours)'
    )

ax.legend(fontsize=12)
ax.set_xlabel('Easting (m)', fontsize=12)
ax.set_ylabel('Northing (m)', fontsize=12)
ax.set_title('FARSITE Simulation: 15 mph Westerly Wind', fontsize=14, fontweight='bold')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Parameter Sweep

In [ ]:
wind_speeds = [5, 10, 15, 20]
results = []

print("Testing different wind speeds...\n")

for ws in wind_speeds:
    print(f"Wind: {ws} mph")
    result = run_farsite(
        ignition_polygon=ignition_poly,
        lcp_path=str(lcp_path),
        windspeed=ws,
        winddirection=270,
        duration_hours=6,
        verbose=False
    )
    results.append((ws, result))
    if result:
        print(f"  → {result.area/1e6:.2f} km²\n")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
axes = axes.flatten()
colors = plt.cm.YlOrRd(np.linspace(0.3, 0.9, len(results)))

for i, (ws, geom) in enumerate(results):
    ax = axes[i]
    
    gpd.GeoDataFrame(geometry=[ignition_poly], crs="EPSG:5070").plot(
        ax=ax, color='red', edgecolor='darkred', linewidth=2
    )
    
    if geom:
        gpd.GeoDataFrame(geometry=[geom], crs="EPSG:5070").plot(
            ax=ax, facecolor=colors[i], edgecolor='black', alpha=0.7, linewidth=2
        )
        area = geom.area / 1e6
    else:
        area = 0
    
    ax.set_title(f'{ws} mph → {area:.2f} km²', fontsize=12, fontweight='bold')
    ax.set_xlabel('Easting (m)')
    ax.set_ylabel('Northing (m)')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle('Fire Growth vs Wind Speed (6 hours)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Done!

Try modifying:
- `fuel_model` in landscape (1=grass, 10=timber, 165=chaparral)
- `winddirection` (0=N, 90=E, 180=S, 270=W)
- `duration_hours` to see longer growth